In [1]:
import numpy as np
import time
from numpy.random import rand
from numpy import exp, log, mat
from numpy.linalg import norm
from scipy.io import loadmat
from scipy.linalg import svd 
from array import array
import copy

In [2]:
def pos(A):
    A[np.where(A<0)] = 0
    return A
def neg(A):
    A[np.where(A>0)] = 0
    return -A

def NNDSVD(A,k):
    if len(A[np.where(A<0)]) > 0:
        print('the input matrix contains negative elements!')
    m,n = A.shape
    
    W = np.zeros((m,k))
    H = np.zeros((k,n))
    
    tmp=svd(A)
    U = tmp[0][:,0:k+1] #3*3
    S = tmp[1][0:k+1]  # 3*1
    V = tmp[2][0:k+1,:] #3*4
    S=np.diag(S)        #3*3
    
    W[:,0] = np.sqrt(S[0,0]) * abs(U[:,0])
    H[0,:] = np.sqrt(S[0,0]) * abs((V[0,:]))

    i_lst=range(2,k+1,1)
    for i in i_lst:
        uu = copy.deepcopy(U[:,i-1])
        vv = copy.deepcopy(V[i-1,:])
        uu1 = copy.deepcopy(U[:,i-1])
        vv1 = copy.deepcopy(V[i-1,:])
        uup = pos(uu)
        uun = neg(uu1) 
        vvp = pos(vv)
        vvn = neg(vv1)
        n_uup = norm(uup)
        n_vvp = norm(vvp) 
        n_uun = norm(uun) 
        n_vvn = norm(vvn) 
        termp = n_uup*n_vvp
        termn = n_uun*n_vvn
        if (termp >= termn):
            W[:,i-1] = np.sqrt(S[i-1,i-1]*termp)*uup/n_uup 
            H[i-1,:] = np.sqrt(S[i-1,i-1]*termp)*vvp.T/n_vvp
        else:
            W[:,i-1] = np.sqrt(S[i-1,i-1]*termn)*uun/n_uun 
            H[i-1,:] = np.sqrt(S[i-1,i-1]*termn)*vvn.T/n_vvn
    W[np.where(W<0.0000000001)]=0.1;
    H[np.where(H<0.0000000001)]=0.1;
    return (W,H)

In [5]:
def objective_function(P,R,S,X,Y,U,V,W,b,lambda1,lambda2,lambda3,lambda4):    
    UW=U.dot(W)
    WtW=W.T.dot(W)
    F=0.5*1.0/(P*S)*norm(X-U.dot(V),'fro')**2+float(lambda1)/P*log(1+exp(-(UW+b)*Y)).sum(axis=0)+0.5*float(lambda2)/R*(WtW+b**2) \
        +0.5*float(lambda3)/(P*R)*norm(U,'fro')**2+0.5*float(lambda4)/(R*S)*norm(V,'fro')**2
    return F
def objective_U(P,R,S,X,Y,U,V,W,b,lambda1,lambda3):
    UW=U.dot(W)
    F_U1 = 0.5*1.0/(P*S)*norm(X-U.dot(V),'fro')**2
    F_U2 = float(lambda1)/P*log(1+exp(-(UW+b)*Y)).sum(axis=0)
    F_U3 = 0.5*float(lambda3)/(P*R)*norm(U,'fro')**2
    F = F_U1+F_U2+F_U3
    #F=0.5*1.0/(P*S)*norm(X-U.dot(V),'fro')**2+float(lambda1)/P*log(1+exp(-(UW+b)*Y)).sum(axis=0) \
    #    +0.5*float(lambda3)/(P*R)*norm(U,'fro')**2
    return F
def subprob_U(P,R,S,X,Y,Uinit,V,W,b,lambda1,lambda3,tol,maxiter):
    U=copy.deepcopy(Uinit)
    VVt=V.dot(V.T)
    XVt=X.dot(V.T)
    YWt=Y.dot(W.T)
    alpha=1.0
    beta=0.1
    iter_lst=range(1,maxiter+1,1)
    ineriter_lst=range(1,21,1)
    for iter in iter_lst:
        UW=U.dot(W)
        Uwby=1+exp((UW+b)*Y)
        grad=1.0/(P*S)*(U.dot(VVt)-XVt)-float(lambda1)/P*(YWt/Uwby)+float(lambda3)/(P*R)*U
        projgrad=norm(np.matrix(grad[np.logical_or(grad<0,U>0)]),'fro')
        print 'Step into U loop\n'
        print '%f\n'%projgrad
        if projgrad<tol:
            print('1')
            print '%f\n'%projgrad
            print '%f\n'%tol
            break
        objold=objective_U(P,R,S,X,Y,U,V,W,b,lambda1,lambda3)
        for ineriter in ineriter_lst:
            Un=U-alpha*grad
            Un[np.where(Un<0)]=0
            d=Un-U
            gradd=(grad*d).sum()
            objnew=objective_U(P,R,S,X,Y,Un,V,W,b,lambda1,lambda3)
            suff_decr=objnew-objold-0.01*gradd<0
            if ineriter==1:
                decr_alpha= not suff_decr
                Up=U
            if decr_alpha:
                if suff_decr:
                    U=Un
                    print('2')
                    break
                else:
                    alpha=alpha*beta
            else:
                if not suff_decr or np.array_equal(Up,Un):
                    U=Up
                    print('3')
                    break
                else:
                    alpha=alpha/beta
                    Up=Un
    if iter==maxiter:
        print('max iter in U\n')
    return U,grad,iter
def objective_W(P,R,S,Y,U,W,b,lambda1,lambda2):
    UW=U.dot(W)
    WtW=W.T.dot(W)
    F=float(lambda1)/P*log(1+exp(-(UW+b)*Y)).sum()+0.5*float(lambda2)/R*(WtW) 
    return F
def subprob_W(P,R,S,Y,U,Winit,b,lambda1,lambda2,tol,maxiter):
    W=copy.deepcopy(Winit)
    alpha=1.0
    beta=0.1
    iter_lst=range(1,maxiter+1,1)
    ineriter_lst=range(1,21,1)
    for iter in iter_lst:
        UW=U.dot(W)
        Uwby=1+exp((UW+b)*Y)
        grad = float(lambda2)/R*W-np.reshape(float(lambda1)/P*((U*Y)/Uwby).sum(axis=0),(-1,1))
        projgrad=norm(grad,'fro')
        print 'Step into W loop\n'
        print '%f\n'%projgrad
        if projgrad<tol:
            print('1')
            print '%f\n'%projgrad
            print '%f\n'%tol
            break
        objold=objective_W(P,R,S,Y,U,W,b,lambda1,lambda2)
        for ineriter in ineriter_lst:
            Wn=W-alpha*grad
            d=Wn-W
            gradd=(grad*d).sum()
            objnew=objective_W(P,R,S,Y,U,Wn,b,lambda1,lambda2)
            suff_decr=objnew-objold-0.01*gradd<0
            if ineriter==1:
                decr_alpha= not suff_decr
                Wp=W
            if decr_alpha:
                if suff_decr:
                    W=Wn
                    print('2')
                    break
                else:
                    alpha=alpha*beta
            else:
                if not suff_decr or np.array_equal(Wp,Wn):
                    W=Wp
                    print('3')
                    break
                else:
                    alpha=alpha/beta
                    Wp=Wn
    if iter==maxiter:
        print('max iter in W\n')
    return W,grad, iter
def objective_b(P,R,S,Y,U,W,b,lambda1,lambda2):    
    UW=U.dot(W)
    F=float(lambda1)/P*log(1+exp(-(UW+b)*Y)).sum()+0.5*float(lambda2)/R*(b**2) 
    return F
def subprob_b(P,R,S,Y,U,W,binit,lambda1,lambda2,tol,maxiter):
    b=copy.deepcopy(binit)
    UW=U.dot(W)
    #Uwby=1+exp((UW+b)*Y)
    alpha=1.0
    beta=0.1
    iter_lst=range(1,maxiter+1,1)
    ineriter_lst=range(1,21,1)
    for iter in iter_lst:
        grad = float(lambda2)/R*b-np.reshape(float(lambda1)/P*(Y/(1+exp((UW+b)*Y))).sum(axis=0),(-1,1))
        projgrad=norm(grad,'fro')
        print 'Step into b loop\n'
        print '%f\n'%projgrad
        if projgrad<tol:
            print('1')
            print '%f\n'%projgrad
            print '%f\n'%tol
            break
        objold=objective_b(P,R,S,Y,U,W,b,lambda1,lambda2)
        for ineriter in ineriter_lst:
            bn=b-alpha*grad
            d=bn-b
            gradd=(grad*d).sum()
            objnew=objective_b(P,R,S,Y,U,W,bn,lambda1,lambda2)
            suff_decr=objnew-objold-0.01*gradd<0
            if ineriter==1:
                decr_alpha=not suff_decr
                bp=b
            if decr_alpha:
                if suff_decr:
                    b=bn
                    print('2')
                    break
                else:
                    alpha=alpha*beta
            else:
                if not suff_decr or np.array_equal(bp,bn):
                    b=bp
                    print('3')
                    break
                else:
                    alpha=alpha/beta
                    bp=bn
    if iter==maxiter:
        print('max iter in b\n')
    return b,grad,iter
def objective_V(P,R,S,X,U,V,lambda4):  
    F_V_1 = 0.5*1.0/(P*S)*norm(X-U.dot(V),'fro')**2
    F_V_2 = 0.5*float(lambda4)/(R*S)*norm(V,'fro')**2
    F = F_V_1 + F_V_2
    #F=0.5*1.0/(P*S)*norm(X-U.dot(V),'fro')**2+0.5*float(lambda4)/(R*S)*norm(V,'fro')**2
    return F
def subprob_V(P,R,S,X,U,Vinit,lambda4,tol,maxiter):
    V=copy.deepcopy(Vinit)
    UtX=U.T.dot(X)
    UtU=U.T.dot(U)
    alpha=1.0
    beta=0.1
    iter_lst=range(1,maxiter+1,1)
    ineriter_lst=range(1,21,1)
    for iter in iter_lst:
        grad=(UtU.dot(V)-UtX)+float(lambda4)*V
        projgrad=norm(np.matrix(grad[np.logical_or(grad<0,V>0)]),'fro')
        print 'Step into V loop\n'
        print '%f\n'%projgrad
        if projgrad<tol:
            print('1')
            print '%f\n'%projgrad
            print '%f\n'%tol
            break
        #objold=objective_V(P,R,S,X,U,V,lambda4)
        for ineriter in ineriter_lst:
            Vn=V-alpha*grad
            Vn[np.where(Vn<0)]=0
            d=Vn-V
            gradd=(grad*d).sum()
            dQd=((UtU.dot(d))*d).sum()
            #objnew=objective_V(P,R,S,X,U,Vn,lambda4)
            #suff_decr=objnew-objold-0.01*gradd<0
            suff_decr=0.99*gradd+0.5*dQd<0
            if ineriter==1:
                decr_alpha=not suff_decr
                Vp=V
            if decr_alpha:
                if suff_decr:
                    V=Vn
                    print('2')
                    break
                else:
                    alpha=alpha*beta
            else:
                if not suff_decr or np.array_equal(Vp,Vn):
                    V=Vp
                    print('3')
                    break
                else:
                    alpha=alpha/beta
                    Vp=Vn
    if iter==maxiter:
        print('max iter in V\n')
    return V,grad,iter
def snmfpg(X, Y, Uinit,Vinit, Winit,binit,lambda1,lambda2,lambda3,lambda4, tol,timelimit,maxiter):
    U = Uinit
    V = Vinit
    W=Winit
    b=binit
    initt=time.time()
    P=U.shape[0]
    R=V.shape[0]
    S=V.shape[1]
    VVt=V.dot(V.T)
    XVt=X.dot(V.T)
    YWt=Y.dot(W.T)
    UW=U.dot(W)
    UtU=U.T.dot(U)
    Uwby=1+exp((UW+b)*Y)
    gradU = 1.0/(P*S)*(U.dot(VVt)-XVt)-float(lambda1)/P*(YWt/Uwby)+float(lambda3)/(P*R)*U
    gradV = 1.0/(P*S)*(UtU.dot(V) - U.T.dot(X))+float(lambda4)/(R*S)*V
    gradW = float(lambda2)/R*W-np.reshape(float(lambda1)/P*((U*Y)/Uwby).sum(axis=0),(-1,1))
    gradb = float(lambda2)/R*b-np.reshape(float(lambda1)/P*(Y/Uwby).sum(axis=0),(-1,1))
    initgrad = norm(gradU,'fro')+norm(gradV,'fro')+norm(gradW,'fro')+norm(gradb,'fro')
    print('Init gradient norm:')
    print '%f\n'%initgrad
    tolU = tol*initgrad
    tolV = tolU
    tolW = tolV
    tolb=tolW

    maxiter_sub = 20;
    obj_total_lst=[]
    for iter in range(1,maxiter+1,1):
        print('Iteration:')
        print '%f\n'%iter
        ngradU = norm(np.matrix(gradU[np.logical_or(gradU<0,U>0)]),'fro');
        ngradV = norm(np.matrix(gradV[np.logical_or(gradV<0,V>0)]),'fro');
        ngradW = norm(gradW,'fro');
        ngradb = norm(gradb,'fro');
        projnorm = ngradU+ngradV+ngradW+ngradb;
        loss_nmf = 0.5*1.0/(P*S)*norm(X-U.dot(V),'fro')**2
        loss_logit = float(lambda1)/P*(log(1+exp(-(U.dot(W)+b)*Y))).sum(axis=0)
        loss_wb = 0.5*float(lambda2)/R*(W.T.dot(W) + b**2)
        loss_U = 0.5*float(lambda3)/(P*R)*norm(U,'fro')**2
        loss_V = 0.5*float(lambda4)/(R*S)*norm(V,'fro')**2
        print('--------------------')
        projnormtol=tol*initgrad
        print '%10.8f\n'%projnorm
        print '%10.8f\n'%projnormtol
        timelimitdiff=time.time()-initt
        print '%10.8f\n'%timelimitdiff
        print '%10.8f\n'%timelimit
        if projnorm<tol*initgrad or time.time()-initt>timelimit:
            print('******************************************************')
            projnormtol=tol*initgrad
            print '%10.8f\n'%projnorm
            print '%10.8f\n'%projnormtol
            timelimitdiff=time.time()-initt
            print '%10.8f\n'%timelimitdiff
            print '%10.8f\nn'%timelimit
            break
        
        U,gradU,iterU = subprob_U(P,R,S,X,Y,U,V,W,b,lambda1,lambda3,tolU,maxiter_sub)
        if iterU==1:
            tolU = 0.1 * tolU  
        W,gradW,iterW = subprob_W(P,R,S,Y,U,W,b,lambda1,lambda2,tolW,maxiter_sub)
        if iterW==1:
            tolW = 0.1 * tolW  
        b,gradb,iterb = subprob_b(P,R,S,Y,U,W,b,lambda1,lambda2,tolb,maxiter_sub)
        if iterb==1:
            tolb = 0.1 * tolb    
        V,gradV,iterV = subprob_V(P,R,S,X,U,V,lambda4,tolV,maxiter_sub+40)
        gradV=1.0/(P*S)*gradV
        if iterV==1:
            tolV = 0.1 * tolV
            
        loss_nmf = 0.5*1.0/(P*S)*norm(X-U.dot(V),'fro')**2
        loss_logit = float(lambda1)/P*(log(1+exp(-(U.dot(W)+b)*Y))).sum(axis=0)
        loss_wb = 0.5*float(lambda2)/R*(W.T.dot(W) + b**2)
        loss_U = 0.5*float(lambda3)/(P*R)*norm(U,'fro')**2
        loss_V = 0.5*float(lambda4)/(R*S)*norm(V,'fro')**2
        tmp=objective_function(P,R,S,X,Y,U,V,W,b,lambda1,lambda2,lambda3,lambda4)
        obj_total_lst.append(tmp)
    return(U,V,W,b,loss_nmf,loss_logit,loss_wb,loss_U,loss_V,obj_total_lst)